In [ ]:
import sys
import json

# files = ['https://storage.googleapis.com/storage/v1/b/mdb-latest/o/ca-british-columbia-translink-vancouver-gtfs-1222.zip?alt=media']
files = ['stl.zip']
# timeseries = [{'start_time': '06:00:00', 'end_time': '08:59:00', 'value': ''}]
timeseries = [
	{'start_time': '06:00:00', 'end_time': '08:59:00', 'value': 'am'},
	{'start_time': '12:00:00', 'end_time': '15:59:00', 'value': 'pm'},
]
params = {'files': files, 'timeseries': timeseries, 'day': 'tuesday', 'dates': []}


default = {'scenario': 'test', 'training_folder': '../..', 'params': params}  # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))

In [2]:
import geopandas as gpd
import pandas as pd
import gtfs_kit as gtk
import numpy as np
from quetzal.io.gtfs_reader import importer
from quetzal.model import stepmodel
import json
import os

In [3]:
# Add path to quetzal
sys.path.insert(0, '../../../../quetzal/')
import os
import numba as nb

on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
num_cores = nb.config.NUMBA_NUM_THREADS

In [4]:
scenario = argv['scenario']
training_folder = argv['training_folder']
# if local. add the path to the scenario scenarios/<scenario>/
if on_lambda:
	input_folder = os.path.join(training_folder, 'inputs/')
	output_folder = os.path.join(training_folder, 'outputs/')
else:
	input_folder = f'../scenarios/{scenario}/inputs/'
	output_folder = f'../scenarios/{scenario}/outputs/'
	num_cores = 4

if not os.path.exists(output_folder):
	os.makedirs(output_folder)


# parameters

In [5]:
argv['params']

{'files': ['stl.zip'],
 'timeseries': [{'start_time': '06:00:00',
   'end_time': '08:59:00',
   'value': 'am'},
  {'start_time': '12:00:00', 'end_time': '15:59:00', 'value': 'pm'}],
 'day': 'tuesday',
 'dates': []}

In [7]:
files = argv['params'].get('files', [])
dates = argv['params'].get('dates', [])
day = argv['params'].get('day', 'monday')
timeseries = argv['params'].get('timeseries', [''])


In [ ]:
temp_period = [t['value'] for t in timeseries]
periods = [f'#{p}' if p != '' else p for p in temp_period]
assert len(periods) == len(set(periods)), 'cannot have 2 periods with the same name'

In [18]:
time_ranges = [[t['start_time'], t['end_time']] for t in timeseries]

In [19]:
print(periods)
print(time_ranges)
assert len(periods) == len(time_ranges)

['#am', '#pm']
[['06:00:00', '08:59:00'], ['12:00:00', '15:59:00']]


In [20]:
DAY_DICT = {'monday': 0, 'tuesday': 1, 'wednesday': 2, 'thursday': 3, 'friday': 4, 'saturday': 5, 'sunday': 6}
selected_day = DAY_DICT[day]

In [21]:
# add input folder if its not a url but a zip file on disc
paths = []
for file in files:
	if file.startswith('http'):
		paths.append(file)
	else:
		paths.append(os.path.join(input_folder, file))


# import

In [22]:
feeds = []
for path in paths:
	print('Importing {f}'.format(f=path))
	feeds.append(importer.GtfsImporter(path=path, dist_units='m'))

Importing ../scenarios/test/inputs/stl.zip


# 1) filling missing values

In [23]:
# 1) filling missing values
for i in range(len(feeds)):
	print('cleaning ', files[i])
	if 'agency_id' not in feeds[i].agency:
		print(f'set agency_id to agency_name in {files[i]}')
		feeds[i].agency['agency_id'] = feeds[i].agency['agency_name']

	if 'agency_id' not in feeds[i].routes:
		print(f'add agency_id to routes in {files[i]}')
		feeds[i].routes['agency_id'] = feeds[i].agency['agency_id'].values[0]

	if 'pickup_type' not in feeds[i].stop_times:
		print(f'pickup_type missing in stop_times. set to 0 in {files[i]}')
		feeds[i].stop_times['pickup_type'] = 0

	if 'drop_off_type' not in feeds[i].stop_times:
		print(f'drop_off_type missing in stop_times. set to 0 in {files[i]}')
		feeds[i].stop_times['drop_off_type'] = 0

	if 'parent_station' not in feeds[i].stops:
		print(f'parent_station missing in stops. set to NaN in {files[i]}')
		feeds[i].stops['parent_station'] = np.nan
	if 'stop_code' not in feeds[i].stops:
		print(f'stop_code missing in stops. set to NaN in {files[i]}')
		feeds[i].stops['stop_code'] = np.nan

	feeds[i].stop_times['pickup_type'].fillna(0, inplace=True)
	feeds[i].stop_times['drop_off_type'].fillna(0, inplace=True)
	feeds[i].stop_times['arrival_time'] = feeds[i].stop_times['departure_time']

cleaning  stl.zip
parent_station missing in stops. set to NaN in stl.zip


# 1) find patterns

In [24]:
for feed in feeds:
	feed.group_services()
	feed.build_stop_clusters(distance_threshold=50)
	feed.build_patterns(on='cluster_id')

# get dates if day is used

In [25]:
# if dates is not provided as inputs. (we have a day)
# get it from first dates of each GTFS
if len(dates) == 0:
	for feed in feeds:
		try:
			min_date = feed.calendar['start_date'].unique().min()
			max_date = feed.calendar['end_date'].unique().max()
		except:
			min_date = feed.calendar_dates['date'].unique().min()
			max_date = feed.calendar_dates['date'].unique().max()

		# get date range
		s = pd.date_range(min_date, max_date, freq='D').to_series()
		try:
			# get dayofweek selected and take first one
			s = s[s.dt.dayofweek == selected_day][0]
			# format  ex: ['20231011'] and append
			dates.append(f'{s.year}{str(s.month).zfill(2)}{str(s.day).zfill(2)}')
		except:
			print('date not available. use', min_date)
			dates.append(min_date)

# 2) restric feed to date

In [26]:
# 2) restric feed to date
feeds_t = []
print('restrict feed')
for i, feed in enumerate(feeds):
	feed_t = feed.restrict(dates=[dates[i]])  # keep time range. will restrict later
	if len(feed_t.trips) > 0:
		feeds_t.append(feed_t)

restrict feed


In [27]:
del feeds

# 3) add shape_dist_traveled to shapes and stop_times

In [28]:
print('add shape_dist_traveled to shapes')
for feed in feeds_t:
	if feed.shapes is None:
		print('no shapes in gtfs')
		continue
	elif 'shape_dist_traveled' not in feed.shapes.columns:
		feed.append_dist_to_shapes()
	elif any(feed.shapes['shape_dist_traveled'].isnull()):
		feed.append_dist_to_shapes()

print('add shape_dist_traveled to stop_times')
for feed in feeds_t:
	if feed.shapes is None:
		print('no shapes in gtfs cannot add to stop_times')
		continue
	elif 'shape_dist_traveled' not in feed.stop_times.columns:
		feed.append_dist_to_stop_times_fast()
	else:
		nan_sequence = feed.stop_times[feed.stop_times['shape_dist_traveled'].isnull()]['stop_sequence'].unique()
		# if there but all nan are at seq=1. just fill wwith 0.
		if all(seq == 1 for seq in nan_sequence):
			feed.stop_times['shape_dist_traveled'] = feed.stop_times['shape_dist_traveled'].fillna(0)
		else:
			feed.append_dist_to_stop_times_fast()

	if feed.stop_times['shape_dist_traveled'].max() < 100:
		print(f'convert to meters')
		feed.dist_units = 'km'
		feed = gtk.convert_dist(feed, new_dist_units='m')

add shape_dist_traveled to shapes
add shape_dist_traveled to stop_times


In [29]:
# nothing to export. export empty geojson
if len(feeds_t) == 0:
	links = gpd.GeoDataFrame(columns=['feature'], geometry='feature', crs=4326)
	nodes = gpd.GeoDataFrame(columns=['feature'], geometry='feature', crs=4326)
	links.to_file(os.path.join(output_folder, 'links.geojson'), driver='GeoJSON')
	nodes.to_file(os.path.join(output_folder, 'nodes.geojson'), driver='GeoJSON')
	end_of_notebook

# 4) build links and nodes.

In [30]:
links_concat = []
nodes_concat = []
for p in range(len(time_ranges)):
	time_range = time_ranges[p]
	links = pd.DataFrame()
	nodes = pd.DataFrame()
	for i in range(len(feeds_t)):
		print('Building links and nodes ', files[i], periods[p])
		feed = feeds_t[i].copy()
		feed_frequencies = feed.convert_to_frequencies(time_range=time_range)
		shapes = feed_frequencies.shapes is not None
		feed_frequencies.build_links_and_nodes(
			log=False,
			shape_dist_traveled=shapes,
			from_shape=shapes,
			stick_nodes_on_links=shapes,
			keep_origin_columns=['departure_time', 'pickup_type'],
			keep_destination_columns=['arrival_time', 'drop_off_type'],
			num_cores=num_cores,
		)
		links = pd.concat([links, feed_frequencies.links.to_crs(4326)])
		nodes = pd.concat([nodes, feed_frequencies.nodes.to_crs(4326)])
	links_concat.append(links)
	nodes_concat.append(nodes)

Building links and nodes  stl.zip #am
Building links and nodes  stl.zip #pm


In [31]:
nodes = pd.concat(nodes_concat)[['stop_id', 'stop_name', 'stop_code', 'geometry']]
nodes = nodes.drop_duplicates(subset=['stop_id'])

# clean columns

In [32]:
columns = [
	'trip_id',
	'route_id',
	'agency_id',
	'direction_id',
	'a',
	'b',
	'shape_dist_traveled',
	'link_sequence',
	'time',
	'headway',
	'pickup_type',
	'drop_off_type',
	'route_short_name',
	'route_type',
	'route_color',
	'geometry',
]

for i in range(len(links_concat)):
	for col in columns:
		if col not in links_concat[i].columns:
			links_concat[i][col] = np.nan
	links_concat[i] = links_concat[i][columns]


In [33]:
# change time = 0 to time = 1 so we dont get infinit speed.
for i in range(len(links_concat)):
	links_concat[i].loc[links_concat[i]['time'] == 0, 'time'] = 1.0

# 5) concat per period (add #period)

In [34]:
# rename time and headway with period (time#am, time#pm)
periods_cols = ['time', 'headway']
for i, p in enumerate(periods):
	period_dict = {col: f'{col}{p}' for col in periods_cols}
	links_concat[i] = links_concat[i].rename(columns=period_dict)


In [35]:
merge_on = ['trip_id', 'a', 'b']
links = links_concat[0]

for new_links in links_concat[1:]:
	links = links.merge(new_links, left_on=merge_on, right_on=merge_on, how='outer', suffixes=('', '_new'))

	for col in columns:
		if col not in merge_on and col in new_links.columns:
			links[col] = links[col].combine_first(links[f'{col}_new'])

	links = links.drop(columns=[c for c in links.columns if c.endswith('_new')])

# fixe time and headway NaN

In [36]:
links[[f'time{p}' for p in periods]] = links[[f'time{p}' for p in periods]].fillna(1)
links[[f'headway{p}' for p in periods]] = links[[f'headway{p}' for p in periods]].fillna(0)

# rename route_type

In [37]:
with open('route_type.json') as file:
	mapping = json.load(file)
	mapping = {int(key): item for key, item in mapping.items()}

retire = ['taxi']
links['route_type'] = links['route_type'].apply(lambda t: mapping.get(t, np.nan))
links = links[~links['route_type'].isin(retire)]


# finish model

In [38]:
sm = stepmodel.StepModel(epsg=4326, coordinates_unit='meter')


In [39]:
sm.links = links
sm.nodes = nodes

sm.nodes = sm.nodes.reset_index(drop=True).sort_index()
sm.links = sm.links.reset_index(drop=True).sort_index()

sm.nodes.loc[sm.nodes['stop_code'].isna(), 'stop_code'] = sm.nodes.loc[sm.nodes['stop_code'].isna(), 'stop_id']

sm.links['trip_id'] = sm.links['agency_id'].astype(str) + '_' + sm.links['trip_id'].astype(str)
sm.links['route_id'] = sm.links['agency_id'].astype(str) + '_' + sm.links['route_id'].astype(str)

sm.links = sm.links.sort_values(['route_type', 'trip_id']).reset_index(drop=True)

dnodes = ('node_' + sm.nodes.reset_index().set_index('stop_id')['index'].astype(str)).to_dict()
sm.nodes.index = 'node_' + sm.nodes.index.astype(str)

sm.links.index = 'link_' + sm.links.index.astype(str)

sm.links['a'] = sm.links['a'].apply(lambda a: dnodes.get(a))
sm.links['b'] = sm.links['b'].apply(lambda a: dnodes.get(a))

sm.links.drop_duplicates(subset=['trip_id', 'link_sequence'], inplace=True)

# Tag route with only one trip
# time_slot = np.diff([hhmmss_to_seconds_since_midnight(time) for time in time_range])[0]
# sm.links.loc[(time_slot/sm.links['headway']) < 2.0, 'headway'] = np.nan

sm.links = sm.links.to_crs(4326)
sm.nodes = sm.nodes.to_crs(4326)


In [40]:
# add speed, add length.
epsg = importer.get_epsg(sm.nodes.iloc[0]['geometry'].y, sm.nodes.iloc[0]['geometry'].x)
sm.links['length'] = sm.links.to_crs(epsg).length
for p in periods:
	sm.links[f'speed{p}'] = (sm.links['length'] / sm.links[f'time{p}']) * 3.6
# regarder quetzal_transit pour voir les valeurs necessaires.

# export

In [41]:
print('Saving')
sm.links.to_file(os.path.join(output_folder, 'links.geojson'), driver='GeoJSON')
sm.nodes.to_file(os.path.join(output_folder, 'nodes.geojson'), driver='GeoJSON')

Saving
